In [51]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import numpy as np
from pyproj import Transformer
from math import sin, cos, radians, atan, degrees
import math

class DEMManagerXYZ:
    """
    Liest und verwaltet DEM-Daten aus einer .xyz-Datei im LV95-Koordinatensystem (EPSG:2056).
    Die Datei enthält Zeilen im Format "X Y Z".
    """
    def __init__(self, dem_path: str, skip_header=False):
        """
        dem_path: Pfad zur .xyz-Datei
        skip_header: True, falls die erste Zeile Header-Text (z.B. "X Y Z") enthält
        """
        self.dem_path = dem_path
        self.skip_header = skip_header

        self.x_coords = None
        self.y_coords = None
        self.z_grid = None

        self.load_dem()

    def load_dem(self) -> None:
        """
        Lädt die .xyz-Daten in ein 2D-Gitter (self.z_grid).
        """
        if not os.path.exists(self.dem_path):
            raise FileNotFoundError(f"DEM-Datei {self.dem_path} wurde nicht gefunden.")

        print(f"Lade .xyz DEM aus '{self.dem_path}'...")

        # Falls die erste Zeile ein Header ist ("X Y Z"), dann skiprows=1
        skiprows = 1 if self.skip_header else 0

        # Einlesen der Zeilen: jede Zeile enthält "X Y Z"
        data = np.loadtxt(self.dem_path, skiprows=skiprows)  # shape = (N, 3)

        xs = data[:, 0]
        ys = data[:, 1]
        zs = data[:, 2]

        # Eindeutige X- und Y-Werte
        unique_x = np.unique(xs)
        unique_y = np.unique(ys)

        unique_x.sort()
        unique_y.sort()

        self.x_coords = unique_x
        self.y_coords = unique_y

        nx = len(unique_x)
        ny = len(unique_y)

        # 2D-Array für Z-Werte
        z_grid = np.full((ny, nx), np.nan, dtype=np.float32)

        # Dictionaries zum schnellen Index-Mapping
        x_to_idx = {val: i for i, val in enumerate(unique_x)}
        y_to_idx = {val: j for j, val in enumerate(unique_y)}

        for x_val, y_val, z_val in data:
            c = x_to_idx[x_val]
            r = y_to_idx[y_val]
            z_grid[r, c] = z_val

        self.z_grid = z_grid
        print(f"DEM geladen. Grid-Dimension: nx={nx}, ny={ny}.")

    def get_elevation(self, x: float, y: float) -> float:
        """
        Gibt die Höhe (Z) an der Position (x, y) (LV95) via bilinearer Interpolation zurück.
        """
        # Prüfen, ob (x, y) im Gültigkeitsbereich liegt
        if x < self.x_coords[0] or x > self.x_coords[-1]:
            raise IndexError("X-Koordinate außerhalb des DEM-Bereichs.")
        if y < self.y_coords[0] or y > self.y_coords[-1]:
            raise IndexError("Y-Koordinate außerhalb des DEM-Bereichs.")

        # Indizes finden
        i1 = np.searchsorted(self.x_coords, x)
        i0 = i1 - 1
        j1 = np.searchsorted(self.y_coords, y)
        j0 = j1 - 1

        # Randfälle
        if i0 < 0:
            i0 = 0
        if i1 >= len(self.x_coords):
            i1 = len(self.x_coords) - 1
        if j0 < 0:
            j0 = 0
        if j1 >= len(self.y_coords):
            j1 = len(self.y_coords) - 1

        x0 = self.x_coords[i0]
        x1 = self.x_coords[i1]
        y0 = self.y_coords[j0]
        y1 = self.y_coords[j1]

        z00 = self.z_grid[j0, i0]
        z01 = self.z_grid[j0, i1]
        z10 = self.z_grid[j1, i0]
        z11 = self.z_grid[j1, i1]

        denom_x = (x1 - x0) if (x1 != x0) else 1e-9
        denom_y = (y1 - y0) if (y1 != y0) else 1e-9

        tx = (x - x0) / denom_x
        ty = (y - y0) / denom_y

        # Bilineare Interpolation
        z0 = z00 + (z01 - z00) * tx
        z1 = z10 + (z11 - z10) * tx
        z = z0 + (z1 - z0) * ty

        return float(z)


class EXIFData:
    """
    Enthält die (teils manuell) gesetzten EXIF-Daten.
    Wichtig ist hier v.a. GPSImgDirection (Heading).
    """
    def __init__(self):
        # Kamera-Parameter (nach Bedarf anpassen)
        self.focal_length_mm = 5.1
        self.sensor_width_mm = 5.76
        self.sensor_height_mm = 4.29
        self.image_width_px = 4032
        self.image_height_px = 3024

        # Kameraposition in WGS84 (Beispielwerte)
        self.latitude = 46.478522
        self.longitude = 8.541992
        self.altitude_m = 2273.53

        # Beispiel: GPS-Heading
        self.gps_img_direction_ref = "T"
        self.gps_img_direction_str = "656565/2048"  # ~ 359.76°
        self.gps_dest_bearing_ref = "T"
        self.gps_dest_bearing_str = "736997/2048"  # ~ 359.76°

        self.gps_img_direction_deg = self._parse_fraction(self.gps_img_direction_str)
        self.gps_dest_bearing_deg = self._parse_fraction(self.gps_dest_bearing_str)

        # Hier definieren wir, wie wir "heading_deg" bestimmen:
        self.heading_deg = self.gps_img_direction_deg

        # Neigung und Roll (hier ungenutzt)
        self.tilt_deg = 0.0
        self.roll_deg = 0.0

    def _parse_fraction(self, fraction_str: str) -> float:
        """
        Erwartet z.B. "736997/2048" und wandelt das in eine float-Zahl um.
        """
        if not fraction_str or '/' not in fraction_str:
            return 0.0
        num_str, den_str = fraction_str.split('/')
        try:
            num = float(num_str)
            den = float(den_str)
            return num / den if den != 0 else 0.0
        except ValueError:
            return 0.0

    def get_decimal_lat_lon(self) -> (float, float):
        """
        Gibt die dezimalen Latitude/Longitude aus.
        """
        return (self.latitude, self.longitude)


class ProjectionManager:
    """
    Verwaltet die Projektion und hier speziell das 3D-Raycasting
    unter Berücksichtigung des Heading und zusätzlicher Pixel-Winkel.
    """
    def __init__(self, dem_manager: DEMManagerXYZ, exif_data: EXIFData):
        self.dem_manager = dem_manager
        self.exif_data = exif_data

        # Kameraposition (WGS84)
        lat_dec, lon_dec = exif_data.get_decimal_lat_lon()
        self.camera_lat = lat_dec
        self.camera_lon = lon_dec
        self.camera_alt = exif_data.altitude_m

        # Heading (z.B. ~359.76° => fast Norden)
        self.heading_deg = exif_data.heading_deg

        # Koordinaten-Transformer
        self.transformer_to_lv95 = Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)
        self.transformer_to_wgs84 = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)

        # Bildabmessungen
        self.image_width_px = exif_data.image_width_px
        self.image_height_px = exif_data.image_height_px

    def get_3d_coordinates_for_pixel(self,
                                 x_pixel: float,
                                 y_pixel: float,
                                 step_size=1.0):
        """
        Bestimmt den 3D-Schnittpunkt auf dem DEM für das Bildpixel (x_pixel, y_pixel).
        Wir brechen ausschließlich ab, wenn:
          - der Strahl die DEM-Oberfläche erreicht (test_z <= dem_h), oder
          - wir aus dem DEM-Bereich hinaustreten (IndexError).
        """
        # 1) Kamera in LV95
        cam_x_lv95, cam_y_lv95 = self.transformer_to_lv95.transform(
            self.camera_lon, self.camera_lat
        )
        cam_z_lv95 = self.camera_alt  # Kamera-Höhe

        # 2) Pixelabhängige Winkel berechnen (Beispielformel)
        zoom_factor = 2.4 # Zoomfaktor der Kamera
        alpha_vert_deg = (-1*degrees(atan(((2*x_pixel / self.image_width_px) - 1.0)* 36.0 / (2.0 * 26.0))))*(1/zoom_factor)  + 12.35 # 12.35° ist der Winkel der Kamera (Tilt), durch Approximationsverfahren ermittelt
        alpha_horiz_deg = (degrees(atan(((2*y_pixel / self.image_height_px) - 1.0) * 24.0 / (2.0 * 26.0))))*(1/zoom_factor)
        print(f"Pixel=({x_pixel}, {y_pixel}) --> alpha_vert={alpha_vert_deg:.2f}°, alpha_horiz={alpha_horiz_deg:.2f}°")

        # 3) In Radianten umwandeln und Heading hinzufügen
        heading_plus_horiz = radians(self.heading_deg + alpha_horiz_deg)
        phi = radians(alpha_vert_deg)  # vertikaler Neigungswinkel

        # 4) 3D-Richtungsvektor
        dx = sin(heading_plus_horiz) * cos(phi)
        dy = cos(heading_plus_horiz) * cos(phi)
        dz = sin(phi)

        # 5) Iteratives Voranschreiten bis Boden oder Austritt aus dem DEM
        dist = 0.0
        intersect_x = cam_x_lv95
        intersect_y = cam_y_lv95
        intersect_z = cam_z_lv95

        iteration = 0
        while True:
            iteration += 1

            test_x = cam_x_lv95 + dx * dist
            test_y = cam_y_lv95 + dy * dist
            test_z = cam_z_lv95 + dz * dist

            # Abfrage der DEM-Höhe für die Testposition
            try:
                dem_h = self.dem_manager.get_elevation(test_x, test_y)
            except IndexError:
                # Außerhalb des DEM-Bereichs -> Kein Schnittpunkt gefunden
                break

            # Jede 10. Iteration ausgeben, inkl. DEM-Höhe
            if iteration % 10 == 0:
                print(f"[Ray Iteration {iteration}] "
                      f"dist={dist:.1f}, x={test_x:.1f}, y={test_y:.1f}, "
                      f"z(Strahl)={test_z:.1f}, DEM-Höhe={dem_h:.1f}")

            # Schnitt mit der Oberfläche?
            if test_z <= dem_h:
                # Schnittpunkt genau auf die DEM-Höhe setzen
                intersect_x = test_x
                intersect_y = test_y
                intersect_z = dem_h
                break

            dist += step_size

        # 6) Rücktransform nach WGS84 (Optional)
        lon_final, lat_final = self.transformer_to_wgs84.transform(intersect_x, intersect_y)
        alt_final = intersect_z

        return lat_final, lon_final, alt_final


def main():
    # 1) DEM (LV95) aus .xyz laden
    dem_path = r".\swissalti3d_2022_2684-1148_0.5_2056_5728.xyz\SWISSALTI3D_0.5_XYZ_CHLV95_LN02_2684_1148.xyz"
    dem_manager = DEMManagerXYZ(dem_path, skip_header=True)

    # 2) EXIF-Daten
    exif_data = EXIFData()
    print("=== Kameraposition (WGS84) ===")
    print(f"Lat={exif_data.latitude:.6f}, Lon={exif_data.longitude:.6f}, Alt={exif_data.altitude_m:.2f}m")
    print(f"GPSImgDirection ~ {exif_data.gps_img_direction_deg:.2f}°")

    # 3) ProjectionManager
    proj_manager = ProjectionManager(dem_manager, exif_data)

    # 4) Drei Beispiel-Pixel
    pixel_positions = [
        (2016, 1512),
        (848, 1267),
        (3828,2576)
    ]

    print("\n=== Berechnungen für Beispiel-Pixel ===")
    for (px, py) in pixel_positions:
        lat_hit, lon_hit, alt_hit = proj_manager.get_3d_coordinates_for_pixel(
            px, py, 
            step_size=1.0
        )
        print(f"Pixel=({px}, {py}) --> lat={lat_hit:.6f}, lon={lon_hit:.6f}, alt={alt_hit:.2f}")


if __name__ == "__main__":
    main()


Lade .xyz DEM aus '.\swissalti3d_2022_2684-1148_0.5_2056_5728.xyz\SWISSALTI3D_0.5_XYZ_CHLV95_LN02_2684_1148.xyz'...
DEM geladen. Grid-Dimension: nx=2000, ny=2000.
=== Kameraposition (WGS84) ===
Lat=46.478522, Lon=8.541992, Alt=2273.53m
GPSImgDirection ~ 320.59°

=== Berechnungen für Beispiel-Pixel ===
Pixel=(2016, 1512) --> alpha_vert=12.35°, alpha_horiz=0.00°
[Ray Iteration 10] dist=9.0, x=2684724.6, y=1148068.6, z(Strahl)=2275.5, DEM-Höhe=2271.9
[Ray Iteration 20] dist=19.0, x=2684718.4, y=1148076.2, z(Strahl)=2277.6, DEM-Höhe=2272.0
[Ray Iteration 30] dist=29.0, x=2684712.2, y=1148083.7, z(Strahl)=2279.7, DEM-Höhe=2273.9
[Ray Iteration 40] dist=39.0, x=2684706.0, y=1148091.3, z(Strahl)=2281.9, DEM-Höhe=2275.4
[Ray Iteration 50] dist=49.0, x=2684699.8, y=1148098.8, z(Strahl)=2284.0, DEM-Höhe=2275.1
[Ray Iteration 60] dist=59.0, x=2684693.6, y=1148106.4, z(Strahl)=2286.1, DEM-Höhe=2274.8
[Ray Iteration 70] dist=69.0, x=2684687.4, y=1148113.9, z(Strahl)=2288.3, DEM-Höhe=2277.0
[Ray Ite